In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数

    data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'turnover_sum', 'change', 'change_sum', 'turnover_N1', 'turnover_N2', 'change_N'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        turn_o = starter['turnover_sum']
        change_s = starter['change_sum']
        differ_t = turn_o.values[1]-turn_o.values[0]                  ###计算T前N1天的成交金额总量
        differ_t2 = turn_o.values[3]-turn_o.values[1]                 ###计算T后N2天的成交金额总量
        differ_c = change_s.values[3]-change_s.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(6,'turnover_N1',[differ_t])
        stock_a.insert(7,'turnover_N2',[differ_t2])
        stock_a.insert(8,'change_N',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)

    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date','stock','close','turnover_N1','turnover_N2','change_N']]
    
    stock_Tmax = stock_f.sort_values(by = ['turnover_N1'], ascending = False).head(K)     ###取出前N1天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change_N'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    

In [4]:
def statisc(data, stock_eg, N1, N2, K):
    
    statis = pd.DataFrame(columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change', 'count'])

    for i in stock_eg['date'].values[N1+1:-N2]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N1, N2, K)
            size = stock_bestinf.iloc[:,0].size                   ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
            avg_turnN1 = stock_bestinf['turnover_N1'].mean()      ###求出满足条件股票的前N1日成交金额平均值
            avg_turnN2 = stock_bestinf['turnover_N2'].mean()      ###求出满足条件股票的后N2日成交金额平均值
            avg_change = stock_bestinf['change_N'].mean()         ###求出满足条件股票的日收益率平均值
            new_data = {'date':stock_bestinf['date'].values[0], 'avg_turnN1':avg_turnN1, 'avg_turnN2':avg_turnN2, 'avg_change':avg_change, 'count':size}
            new_add = pd.DataFrame(new_data, columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change', 'count'],index=[0])     ###将列表合并
            statis = statis.append(new_add, ignore_index=True)
        except:
            continue

    return statis

In [5]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2017/2017.csv',encoding='gbk',engine='python')

In [6]:
stock_m = 1
#stock_m = max_stock(data)
stock_eg = data[data['stock'] == stock_m]

In [7]:
statis = statisc(data, stock_eg, 55, 15, 100)
statis

,date,avg_turnN1,avg_turnN2,avg_change,count
0,2017-03-29,3.689459e+10,1.993857e+10,0.011290,6
1,2017-03-30,4.090497e+10,2.345789e+10,0.010607,5
2,2017-03-31,4.284356e+10,2.297017e+10,0.008470,4
3,2017-04-05,4.217499e+10,2.143128e+10,0.010303,4
4,2017-04-06,4.312304e+10,1.977157e+10,0.009938,5
5,2017-04-07,4.665945e+10,2.094880e+10,0.010506,6
6,2017-04-10,4.668238e+10,2.016848e+10,0.010876,5
7,2017-04-11,3.950932e+10,2.034007e+10,0.013135,3
8,2017-04-12,5.279726e+10,2.256627e+10,0.008771,7
9,2017-04-13,5.086085e+10,2.210264e+10,0.010276,9


In [8]:
statis.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017/2017-55-15.csv', index=False)